In [1]:
!pip install basemap
!pip install basemap-data-hires
!pip install geopandas
!pip install openturns

  Obtaining dependency information for basemap from https://files.pythonhosted.org/packages/f7/0e/1c3e4b5b4f6e59075e51252ac3a6a03e3cc59a15d67bf08e2008418ed69b/basemap-1.3.8-cp38-cp38-manylinux1_x86_64.whl.metadata
  Using cached basemap_data-1.3.2-py2.py3-none-any.whl (30.5 MB)
  Using cached pyshp-2.3.1-py2.py3-none-any.whl (46 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 6.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.3/865.3 kB 5.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Using cached basemap_data_hires-1.3.2-py2.py3-none-any.whl (91.1 MB)
  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/cb/7b/2fb6ab376c78d2e60812ad0bf4b24889309d2aeb543f54efcd586c69907f/geopandas-0

In [22]:
# from google.colab import drive, files
# drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd
import math
import copy
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
from scipy.spatial import ConvexHull
from timeit import default_timer as timer
from datetime import datetime
from datetime import timedelta
import openturns as ot
from openturns.viewer import View

In [3]:
file_name = "aisdk_20180101"
# date_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/TOPOA/AIS data/denmark ais/dk_csv_20180101/aisdk_20180101.csv", encoding='unicode_escape')
date_df = pd.read_csv("./dk_csv_20180101/{}.csv".format(file_name), encoding='unicode_escape')

In [3]:
date_df

NameError: name 'date_df' is not defined

In [5]:
def get_cartesian(lat=None,lon=None):
    # https://stackoverflow.com/a/55256861
    lat, lon = np.deg2rad(lat), np.deg2rad(lon)
    R = 6371 # radius of the earth (km)
    x = R * np.cos(lat) * np.cos(lon) * 1000 # (m)
    y = R * np.cos(lat) * np.sin(lon) * 1000 # (m)
    # z = R *np.sin(lat)
    return x,y

## Data pre-procssing1 - GPS conversion

### parameters

In [6]:
# Boundaries
km_to_m = 1000 # if use km put 1


left_x = 2000 * km_to_m
right_x = 4000 * km_to_m
upper_y = 700 * km_to_m
lower_y = 300 * km_to_m
spd_constraint = 0.2
encounter_dist = 2 * km_to_m
sampling_time = 15 # seconds
effective_encounter_dist = 3 * 1852 # nm 
clearing_cond_dist = 1 * 1000 # 1 km 
clearing_cond_angle = 112.5 # deg

finish_finding_ships = 200
bow_crossing_hit_time = 3 # ea

In [7]:
gps_processed_df = date_df.copy()
gps_processed_df['pos_x'], gps_processed_df['pos_y'] = get_cartesian(date_df['Latitude'], date_df['Longitude'])

In [8]:
gps_processed_df.columns = gps_processed_df.columns.str.replace('# ', '')

In [9]:
gps_processed_df.head()

,Timestamp,Type of mobile,MMSI,Latitude,Longitude,Navigational status,ROT,SOG,COG,Heading,...,Cargo type,Width,Length,Type of position fixing device,Draught,Destination,ETA,Data source type,pos_x,pos_y
0,01/01/2018 00:00:00,Base Station,2194006,55.538917,5.033400,Unknown value,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Surveyed,NaN,NaN,NaN,AIS,3.591105e+06,316290.515182
1,01/01/2018 00:00:00,Base Station,2194005,56.344267,4.272000,Unknown value,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Surveyed,NaN,NaN,NaN,AIS,3.521008e+06,263015.557728
2,01/01/2018 00:00:00,Class A,219001061,55.716887,12.593578,Under way using engine,0.0,0.0,78.8,20.0,...,NaN,NaN,NaN,Undefined,NaN,NaN,NaN,AIS,3.502334e+06,782452.227317
3,01/01/2018 00:00:00,Class A,236407000,57.423072,8.690758,Under way using engine,-3.6,11.6,51.5,53.0,...,NaN,NaN,NaN,Undefined,NaN,NaN,NaN,AIS,3.390961e+06,518330.148107
4,01/01/2018 00:00:00,Class A,265513250,56.157983,15.594550,Under way using engine,NaN,0.0,19.3,NaN,...,NaN,NaN,NaN,Undefined,NaN,NaN,NaN,AIS,3.417431e+06,953813.442097


In [10]:
# crop the region and non-valid value
gps_processed_df = gps_processed_df.loc[(right_x >= gps_processed_df['pos_x']) 
                                        & (gps_processed_df['pos_x'] >= left_x) 
                                        & (upper_y >= gps_processed_df['pos_y'])
                                        & (gps_processed_df['pos_y'] >= lower_y)].dropna(subset=['Timestamp','MMSI', 'Heading', 'SOG', 'COG'])

In [11]:
gps_processed_df['Timestamp'] =  pd.to_datetime(gps_processed_df['Timestamp'])
gps_processed_df['Timestamp'] =  gps_processed_df['Timestamp'].dt.strftime('%d/%m/%y %H:%M:%S')
# gps_processed_df['Timestamp']  = pd.to_datetime(gps_processed_df['Timestamp'], format='%d/%m/%y %H:%M:%S')

* trajectory linear interpolation
* one ship --> other ships perspective

In [35]:
i = 0
j = 0


found = False
df_entire_pass = pd.DataFrame()

for ego_id, df_by_ego_ID in gps_processed_df.groupby('MMSI'):
    # -------------------------------------------------------------------------------
    # ego vehicle
    # -------------------------------------------------------------------------------
    # edge case where it is not using engine
    if (df_by_ego_ID['Navigational status'] != 'Under way using engine').any():
        continue

    # 1) timestamp reindex for interpolate
    df_by_ego_ID['Timestamp'] =  pd.to_datetime(df_by_ego_ID['Timestamp'], format='%d/%m/%y %H:%M:%S')
    df_by_ego_ID = df_by_ego_ID.set_index('Timestamp')

    # 2) filling the gap
    df_ego_interpol = df_by_ego_ID.resample("{}s".format(str(sampling_time))).mean(numeric_only=True) # mean or padding any other method?
    df_ego_interpol = df_ego_interpol.interpolate() 
    # TODO linear or something? https://pandas.pydata.org/docs/reference/api/pandas.core.resample.Resampler.interpolate.html
    # method='spline'

    # print(df_ego_interpol)


    # -------------------------------------------------------------------------------
    # other vehicle
    # -------------------------------------------------------------------------------
    # TODO
    # 1) make a function
    for obj_id, df_by_obj_ID in gps_processed_df.groupby('MMSI'):
        if j >= finish_finding_ships:
            break

        # edge case: not ego vehicle
        if obj_id == ego_id:
            continue

        # 1) timestamp reindex for interpolate
        df_by_obj_ID['Timestamp'] =  pd.to_datetime(df_by_obj_ID['Timestamp'], format='%d/%m/%y %H:%M:%S')
        df_by_obj_ID = df_by_obj_ID.set_index('Timestamp')

        # 2) filling the gap
        df_obj_interpol = df_by_obj_ID.resample("{}s".format(str(sampling_time))).mean(numeric_only=True) # mean or padding any other method?
        df_obj_interpol = df_obj_interpol.interpolate() # TODO linear or something?

        # 3) find intersection
        # https://sparkbyexamples.com/pandas/find-intersection-between-two-series-in-pandas/?expand_article=1
        # time_intersect = set(df_ego_interpol.index) & set(df_obj_interpol.index)
        time_intersect = df_ego_interpol.index.intersection(df_obj_interpol.index)

        # 4) find closest distance meeting
        if len(time_intersect):
            ############################# 
            # crop data processing 
            #############################

            # TODO rel speed
            df_rel_obj = pd.DataFrame()
            df_rel_obj['x'] = df_ego_interpol.loc[time_intersect]['pos_x'] - df_obj_interpol.loc[time_intersect]['pos_x']
            df_rel_obj['y'] = df_ego_interpol.loc[time_intersect]['pos_y'] - df_obj_interpol.loc[time_intersect]['pos_y']
            df_rel_obj['v_x'] = df_rel_obj['x'].diff() / sampling_time # relative vel x
            df_rel_obj['v_y'] = df_rel_obj['y'].diff() / sampling_time # relative vel y

            df_rel_obj['rel_dist'] = np.sqrt(df_rel_obj['x']**2 + df_rel_obj['y']**2)
            df_rel_obj['rel_bearing'] = np.arctan2(df_rel_obj['y'], df_rel_obj['x']) # atan wrt my vehicle
            df_rel_obj['rel_bearing_diff'] = df_rel_obj['rel_bearing'].diff()
            df_rel_obj['Heading'] = df_obj_interpol['Heading']
            df_rel_obj['Heading_by_xy'] = np.arctan2(df_obj_interpol.loc[time_intersect]['pos_y'].diff(), df_obj_interpol.loc[time_intersect]['pos_x'].diff())

            # within distance threshold
            encounter_cond = (df_rel_obj['rel_dist'] <= encounter_dist).any()

            # meaning encounter as below a certain distance  
            if encounter_cond: 
                ########################################################## 
                # 1) valid encounter find: approaching from the front side
                ##########################################################
                """ encounter detail condition crop """
                df_rel_obj_crop = df_rel_obj.loc[(df_rel_obj['rel_dist'] <= effective_encounter_dist)] # TODO TCPA-based, as this can address each vessel character
                # print("cropped", df_rel_obj_crop)
                end_idx = 0

                for k in range(len(df_rel_obj_crop)):
                    rel_dist = df_rel_obj_crop['rel_dist'].iloc[k]
                    rel_bearing = df_rel_obj_crop['rel_bearing'].iloc[k]

                    """ clearing condition """
                    if rel_dist >= clearing_cond_dist and \
                        abs(rel_bearing) >= math.radians(clearing_cond_angle):
                        # print("here enter")
                        end_idx = k
                        break
                # print("k value {}".format(end_idx))

                # TODO by function
                if end_idx != 0:
                    # 1) pos_x extraction to extract coming from front
                    df_rel_obj_crop = df_rel_obj_crop.iloc[:end_idx]

                    init_pos_x = df_rel_obj_crop['x'].iloc[0]
                    last_pos_x = df_rel_obj_crop['x'].iloc[-1]

                    # TODO backpassing (overtaking)
                    if init_pos_x <= 0: 
                        # df.loc[(df.obj_ID == obj_id), 'valid'] = False
                        # df_rel_obj_crop['valid'] = False
                        continue
                    
                    # coming from the front
                    else:
                        # TODO or passing my stern line 
                        if last_pos_x <= 0: # ended up passing 
                            # df.loc[(df.obj_ID == obj_id), 'valid'] = True
                            df_rel_obj_crop['valid'] = True
                            df_rel_obj_crop['obj_index'] = j
                            found = True

                            
                        else:
                            # df.loc[(df.obj_ID == obj_id), 'valid'] = False
                            # df_rel_obj_crop['valid'] = False
                            continue
                            
                    print("MMSI {} found valid ....... ".format(obj_id))

                    ########################################################## 
                    # 2) angle difference, r, and winding number
                    ##########################################################

                    # passing homotopy
                    winding_number = df_rel_obj_crop['rel_bearing_diff'].sum()
                    if winding_number > 0:
                        # print("passing on my L side")
                        df_rel_obj_crop['label'] = 'L' # entire cell mapping

                        #  # Bow crossing from 1st quadrant to 4th, 3rd  
                        # if len(df_rel_obj_crop.loc[(df_rel_obj_crop['y'] < 0) & (df_rel_obj_crop['x'] > 0)]) >= bow_crossing_hit_time:
                        #     df_rel_obj_crop['bow_crossing'] = True
                        # else:
                        #     df_rel_obj_crop['bow_crossing'] = False
                    
                    
                    elif winding_number < 0:
                        # print("passing on my R side")
                        df_rel_obj_crop['label'] = 'R' # entire cell mapping

                        # # Bow crossing from 4th quadrant to 1st, 2nd  
                        # if len(df_rel_obj_crop.loc[(df_rel_obj_crop['y'] > 0) & (df_rel_obj_crop['x'] > 0)]) >= bow_crossing_hit_time:
                        #     df_rel_obj_crop['bow_crossing'] = True
                        # else:
                        #     df_rel_obj_crop['bow_crossing'] = False

                    # bow crossing
                    # Bow crossing from 1st quadrant to 4th, 3rd  
                    # if (df_rel_obj_crop['label'] == 'R').all() and \
                    # len(df_rel_obj_crop.loc[(df_rel_obj_crop['y'] > 0) & (df_rel_obj_crop['x'] > 0)]) >= bow_crossing_hit_time:
                    #     df_rel_obj_crop['bow_crossing'] = True

                    # # Bow crossing from 4th quadrant to 1st, 2nd  
                    # elif (df_rel_obj_crop['label'] == 'L').all() and\
                    # len(df_rel_obj_crop.loc[(df_rel_obj_crop['y'] < 0) & (df_rel_obj_crop['x'] > 0)]) >= bow_crossing_hit_time:
                    #     df_rel_obj_crop['bow_crossing'] = True

                    # else:
                    #     df_rel_obj_crop['bow_crossing'] = False
                    
                    print("MMSI {} found to pass: {} and bow crossing: {} ".format(obj_id, df_rel_obj_crop['label'][0], df_rel_obj_crop['bow_crossing'][0]))

                    df_rel_obj_crop.to_pickle("./dk_csv_20180101/{}_{}.pkl".format(file_name, j))
                    print("{} th object saved ....... {}_{}.pkl".format(j, file_name, j))
                    print("-------------------------------------------------------------")

                    # merge current object with entire dataframe
                    df_entire_pass = pd.concat([df_entire_pass, df_rel_obj_crop])
                    
                    j += 1

    if j >= finish_finding_ships:
        break

print("final {}".format(j))

MMSI 220487000 found valid ....... 
MMSI 220487000 found to pass: L and bow crossing: True ....... 
0 th object saved ....... aisdk_20180101_0.pkl
MMSI 266266000 found valid ....... 
MMSI 266266000 found to pass: L and bow crossing: True ....... 
1 th object saved ....... aisdk_20180101_1.pkl
MMSI 244850087 found valid ....... 
MMSI 244850087 found to pass: L and bow crossing: False ....... 
2 th object saved ....... aisdk_20180101_2.pkl
MMSI 357562000 found valid ....... 
MMSI 357562000 found to pass: L and bow crossing: True ....... 
3 th object saved ....... aisdk_20180101_3.pkl
MMSI 477967700 found valid ....... 
MMSI 477967700 found to pass: L and bow crossing: True ....... 
4 th object saved ....... aisdk_20180101_4.pkl
MMSI 245655000 found valid ....... 
MMSI 245655000 found to pass: R and bow crossing: True ....... 
5 th object saved ....... aisdk_20180101_5.pkl
MMSI 230352000 found valid ....... 
MMSI 230352000 found to pass: L and bow crossing: True ....... 
6 th object saved

In [ ]:
unpickled_df = pd.read_pickle("./dk_csv_20180101/aisdk_20180101_214.pkl")
unpickled_df